= Лабораторная работа 3: RNN, GRU, LSTM =

БВТ2202, Градов Артём Николаевич

Цель - написать RNN/GRU/LSTM модель которая будет способна классифицировать текст на токсичный или не токсичный.

В чём разница между RNN, GRU и LSTM?
RNN = Простейшая рекуррентная сеть с одним скрытым состоянием
GRU (Gated Recurrent Unit) = Улучшенная RNN с двумя гейтами (update и reset). Меньше параметров, чем у LSTM, лучше запоминает контекст
LSTM (Long Short-Term Memory) = Содержит три гейта (input, forget, output) и cell state.Отлично справляется с long-term dependencies.

LSTM и GRU решают проблему vanishing gradients через механизм гейтов, притом LSTM сложнее и мощнее, а GRU — более легковесная альтернатива.

Необходимо использовать токенизатор с Hugging Face - разбиваем текст на токены и преобразуем их в ID, которые можно подать в эмбединг-слой.

Также Loss BCE (Binary Cross-Entropy) и оптимизатор Adam/AdamW при learning rate=3e-4. AdamW почти всегда лучше, потому что он исправляет проблему L2-регуляризации в Adam и корректно отделяет decay от градиентов

In [25]:
import torch
from torch import nn, optim, tensor
import torchvision
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Используемое устройство:", device)

Используемое устройство: cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data = pd.read_csv('/content/drive/MyDrive/labeled.csv')
texts = data['comment'].values
labels = data['toxic'].values.astype(float)

print(texts[0], labels[0])

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.15, random_state=42
)

Верблюдов-то за что? Дебилы, бл...
 1.0


In [7]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
length = 128  #ограничение для оптимизации, меньше padding -> меньше вычислений

def tokenize(texts):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=length,
        return_tensors="pt"
    )

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
      return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'label': torch.tensor(self.labels[idx], dtype=torch.float)
        }
        return item

train_dataset = ToxicDataset(train_encodings, train_labels)
test_dataset = ToxicDataset(test_encodings, test_labels)

In [15]:
class LSTMmodel(nn.Module):
  def __init__(self, size, embedding_dim=300, hidden_dim=256):
    super().__init__()
    self.embedding = nn.Embedding(size, embedding_dim)
    self.lstm = nn.LSTM(
      embedding_dim,
      hidden_dim,
      batch_first=True,
      bidirectional=True
    )
    self.classifier = nn.Sequential(
        nn.Linear(hidden_dim*2,64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,1),
        nn.Sigmoid()
    )

  def forward(self, input_id, attention_mask=None):
    x = self.embedding(input_id)
    lstm_out, _ = self.lstm(x)
    out = lstm_out[:, -1, :]
    return self.classifier(out)

In [28]:
v_size = tokenizer.vocab_size
model = LSTMmodel(v_size)
model.to(device)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
criterion = nn.BCELoss()

def trainEpoch(model, dataloader):
  model.train()
  loss = 0
  for b in dataloader:
    optimizer.zero_grad()
    input_id = b['input_ids'].to(device)
    labels = b['label'].to(device)

    out = model(input_id)

    loss_by_batch = criterion(out.squeeze(), labels)
    loss_by_batch.backward()
    optimizer.step()
    loss += loss_by_batch.item()

  return loss / len(dataloader)

def evaluation(model, dataloader):
  model.eval()
  preds = []
  probs = []
  labels = []

  with torch.no_grad():
    for b in dataloader:
      input_id = b['input_ids'].to(device)
      b_labels = b['label'].to(device)

      out = model(input_id)
      b_probs = out.squeeze().cpu().numpy()
      b_preds = (b_probs > 0.5).astype(int)

      probs.extend(b_probs)
      preds.extend(b_preds)
      labels.extend(b_labels.cpu().numpy())

    acc = np.mean(np.array(preds)==np.array(labels))
    f1 = f1_score(labels, preds)
    roc_auc = roc_auc_score(labels, probs)
    pr_auc = average_precision_score(labels, probs)

    return [acc, f1, roc_auc, pr_auc]

In [30]:
n_epochs = 5
for e in range(n_epochs):
  loss = trainEpoch(model, train_loader)
  res = evaluation(model, test_loader)
  print("EPOCH", e, "/", n_epochs-1)
  print(f"Train Loss: {loss:.4f}, Accuracy: {res[0]*100:.6f}%")
  print(f"F1: {res[1]:.4f}, ROC-AUC: {res[2]:.4f}, PR-AUC: {res[3]:.4f}")

EPOCH 0 / 4
Train Loss: 0.3809, Accuracy: 78.769658%
F1: 0.6990, ROC-AUC: 0.8344, PR-AUC: 0.6222
EPOCH 1 / 4
Train Loss: 0.3074, Accuracy: 83.348751%
F1: 0.7443, ROC-AUC: 0.8782, PR-AUC: 0.7585
EPOCH 2 / 4
Train Loss: 0.2709, Accuracy: 82.747456%
F1: 0.7368, ROC-AUC: 0.8721, PR-AUC: 0.7401
EPOCH 3 / 4
Train Loss: 0.2236, Accuracy: 82.654949%
F1: 0.7508, ROC-AUC: 0.8753, PR-AUC: 0.7245
EPOCH 4 / 4
Train Loss: 0.2197, Accuracy: 82.932470%
F1: 0.7502, ROC-AUC: 0.8838, PR-AUC: 0.7561


F1 - Баланс между precision и recall (важно при дисбалансе классов)

ROC-AUC - Площадь под ROC-кривой (чувствительность к ранжированию)

PR-AUC - Площадь под Precision-Recall кривой (важно для дисбалансированных данных)